<a href="https://colab.research.google.com/github/xchen224/skorch/blob/main/cloud_datapro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
####
def tropomi_l1brad_to_refl(l1bdata,
                     bandname,
                     region=[-180,180,-90,90],
                     irr_filename=None,
                     channel=None):
  """
  Convolve tropomi L1B radiance to reflectance 
  """

  mask3d, mask = tropomi_l1b_prescreen(l1bdata['radiance'],
                                         l1bdata['latitude'],
                                         l1bdata['longitude'], region,
                                         l1bdata['measurement_quality'],
                                         l1bdata['ground_pixel_quality'],
                                         l1bdata['spectral_channel_quality'])

  npixel = l1bdata['radiance'].shape[2]
  nchannel = l1bdata['radiance'].shape[3]
  nscan = l1bdata['radiance'].shape[1]

  print(npixel, nchannel, nscan)
  #print(mask[1265:1267,150], type(mask[1265:1267,150]))
  mask0 = np.zeros(mask.shape)
  mask0[:] = False

  # define output data
  theta0 = l1bdata['solar_zenith_angle'][0, :, :]
  theta0.mask = mask
  theta = l1bdata['viewing_zenith_angle'][0, :, :]
  theta.mask = mask
  phi0 = l1bdata['solar_azimuth_angle'][0, :, :]
  phi0.mask = mask
  phi = l1bdata['viewing_azimuth_angle'][0, :, :]
  phi.mask = mask
  lat = l1bdata['latitude'][0, :, :]
  lat.mask = mask0
  lon = l1bdata['longitude'][0, :, :]
  lon.mask = mask0
  
  time0 = np.reshape(np.tile(l1bdata['time'][0, :], npixel), (npixel, nscan))
  time = time0.T
  time.mask = mask0

  radiance = l1bdata['radiance'][0, :, :, :]
  radiance.mask = mask3d

  if irr_filename is None:
    print('Error: TROPOMI solar irradiance filename is not defined!')
    sys.exit()
  else:
    irr, invalid0 = read_tropomi_irr(irr_filename, channel)
    invalid = np.ma.filled(invalid0, fill_value=True)

  # pifactor 3d
  theta0_1 = np.empty((theta0.shape[0],theta0.shape[1],1))
  theta0_1[:,:,0] = theta0
  theta0_3d  = np.tile( theta0_1, nchannel )
  mask_1   = np.empty((theta0.shape[0],theta0.shape[1],1))
  mask_1[:,:,0] = mask
  mask_angle = np.tile( mask_1, nchannel )
  theta0_3d  = np.ma.masked_array( theta0_3d, mask=mask_angle )
  print('check theta0 3d:', theta0_3d[:,:,0], theta0)
  
  pifactor = np.pi / np.cos(theta0_3d * np.pi / 180.)

  # refl = rad / E0 * pifactor
  irr3d = np.tile( irr['irradiance'], (nscan,1,1) )
  invalid3d = np.tile( invalid, (nscan,1,1) )
  irr3d.mask = invalid3d
  print('check irr 3d:', irr3d.shape, invalid3d.shape)

  refl = radiance / irr3d * pifactor

  return dict(rad=radiance,
                refl=refl,
                lat=lat,
                lon=lon,
                theta0=theta0,
                theta=theta,
                phi0=phi0,
                phi=phi,
                time=time)
